In [ ]:
!pip install pytorch-ignite
!pip install torch-lr-finder
!wget https://xane-data.s3.ap-south-1.amazonaws.com/datasets/xane-ai/seefood.tgz

In [ ]:
!tar -xvzf './seefood.tgz'

In [ ]:
!rm './seefood.tgz'
!rm './seefood/.DS_Store'
!rm './seefood/._.DS_Store'

In [ ]:
import numpy as np
import torch
from torch_lr_finder import LRFinder
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image, ImageDraw
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
import os
import cv2
import torch.optim as optim
from ignite.engine import create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss, Accuracy
from ignite.contrib.handlers import FastaiLRFinder, ProgressBar

In [ ]:
path = "seefood"
for folder in os.listdir(path):
  for img_file in os.listdir(os.path.join(path,folder)):
    img_file = os.path.join(path,folder,img_file)
    try:
      img = Image.open(img_file)
      if img.mode!='RGB':
        os.remove(img_file)
        print(img_file)
    except:
      print(img_file)
      os.remove(img_file)

In [ ]:
transform = transforms.Compose([
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5],[0.5])
                              ])
transform2 = transforms.Compose([
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5],[0.5]),
                                transforms.RandomRotation((0,90)),
                                transforms.Grayscale(3)
                              ])


In [ ]:
dataset1 = datasets.ImageFolder('seefood',transform=transform)
dataset2 = datasets.ImageFolder('seefood',transform=transform2)
dataset = dataset1 + dataset2
dataset_len = len(dataset)

In [ ]:
train_len,test_len = dataset_len - 200,200
train_set,test_set = torch.utils.data.random_split(dataset,[train_len,test_len],generator=torch.manual_seed(20))
batch_size = 100
train_set = DataLoader(dataset=train_set,shuffle=True,batch_size=batch_size)
test_set = DataLoader(dataset=test_set,shuffle=True,batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using Device: ', device)

Using Device:  cpu


In [ ]:
model = models.resnet18(pretrained=True)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


In [ ]:
for param in model.parameters(): 
  param.requires_grid = False 
  model.fc = torch.nn.Linear(in_features=512,out_features=2,bias=True) 
  model = model.to(device)

In [ ]:
list(model.parameters())[0].shape

torch.Size([64, 3, 7, 7])

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(),lr=5.34e-4,weight_decay=10e-5)


In [ ]:
# lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
# lr_finder.range_test(train_set,start_lr=10e-5, end_lr=1, num_iter=100)
# lr_finder.plot() # to inspect the loss-learning rate graph
# lr_finder.reset()

In [ ]:
model.train()
for epoch in range(10):
  total_correct = 0.0
  running_loss = 0.0
  for i,(inputs,labels) in enumerate(train_set):
    inputs,labels = inputs.to(device),labels.to(device)
    output = model(inputs)
    output_idx = torch.argmax(output,dim=1)
    total_correct+=(labels == output_idx).sum().item()
    optimizer.zero_grad()
    loss = criterion(output,labels)
    running_loss+= loss.item()*inputs.size(0)
    loss.backward()
    optimizer.step()
  print(f'Epoch: {epoch} Loss = {running_loss/train_len} Accuracy:{(total_correct/train_len)*100}%')

In [ ]:
with torch.no_grad():
  model.eval()
  total_loss = 0.0
  total_correct = 0.0
  for inputs,labels in test_set:
    labels = labels.to(device)
    outputs = model(inputs.to(device))
    loss = criterion(outputs,labels)
    total_loss = loss.item() * inputs.size(0)
    output_idx = torch.argmax(outputs,dim=1)
    total_correct+=sum(labels == output_idx)
print(f'Loss = {running_loss/test_len} Accuracy:{(total_correct/test_len)*100}%')

In [ ]:
torch.save(model.state_dict(),'hot_dog_vs_not_hot_dog.pt')

In [ ]:
import numpy as np
import torch
from torchvision import models,transforms
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
labels = ['hot_dog','not_hot_dog']
model_path = 'hot_dog_vs_not_hot_dog.pt'
img_file_path = '/content/download.jpg'
transform = transforms.Compose([transforms.CenterCrop(224),transforms.ToTensor()])
model = models.resnet18()
model.fc = torch.nn.Linear(in_features=512,out_features=len(labels),bias=True)
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
model.eval()
img = Image.open(img_file_path)
img = transform(img)
prediction = model(img.unsqueeze(0))
print(prediction)
result = torch.argmax(prediction)
print(result)
print(f'This is a {labels[result]}')

In [ ]:
import torch
from torchvision import models

In [ ]:
pt_model = models.resnet18()
pt_model.fc = torch.nn.Linear(in_features=512,out_features=2,bias=True)
pt_model.load_state_dict(torch.load('hot_dog_vs_not_hot_dog.pt',map_location=torch.device('cpu')))

dummy_input = torch.randn(64,3,224,224,device='cpu')
torch.onnx.export(pt_model,               # model being run
                  dummy_input,                         # model input (or a tuple for multiple inputs)
                  "/content/onnx_model_hotdog.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [ ]:
!pip install onnx
!pip install onnxruntime

In [ ]:
import numpy
import onnxruntime
import onnx

In [ ]:
onnx_model = onnx.load("/content/onnx_model_hotdog.onnx")
onnx.checker.check_model(onnx_model)


In [ ]:
ort_session = onnxruntime.InferenceSession("/content/onnx_model_hotdog.onnx")
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [ ]:
img_file_path = '/content/download.jpg'
transform = transforms.Compose([transforms.CenterCrop(224),transforms.ToTensor()])
img = Image.open(img_file_path)
img = transform(img)
img = img.unsqueeze(0)
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img)}
ort_outs = ort_session.run(None, ort_inputs)[0]

In [ ]:
np.argmax(ort_outs)